# Two guys meet

This example is intended to illustrate the use of `Simulate.jl`.

In [1]:
using Simulate, Printf

Since we implement the encounter as state machines, we need some definitions. For simplicity we don't implement a separate state here but only events.

In [2]:
struct Guy
    name
end

abstract type Encounter end
struct Meet <: Encounter
    someone
end
struct Greet <: Encounter
    num
    from
end
struct Response <: Encounter
    num
    from
end

We have some convenience functions. `τ()` gives the current simulation time.

In [3]:
comm = ("Nice to meet you!", "How are you?", "Have a nice day!", "bye bye")
say(name, n) =  @printf("%5.2f s, %s: %s\n", τ(), name, comm[n])

say (generic function with 1 method)

Now we implement the actions between the state machines as different methods of a `step!`-function, which will be called according to events. Here we use three `Sim.jl`-features:

- italic `𝐶` (`\itC`+Tab) or `Clk` is the central clock,
- `SimFunction` gives us the possibility to store a function for later execution,
- `event!` schedules our `SimFunction` for an execution after some time. 

In [4]:
function step!(me::Guy, σ::Meet)
    event!(𝐶, SimFunction(step!, σ.someone, Greet(1, me)), after, 2*rand())
    say(me.name, 1)
end

function step!(me::Guy, σ::Greet)
    if σ.num < 3
        event!(𝐶, SimFunction(step!, σ.from, Response(σ.num, me)), after, 2*rand())
        say(me.name, σ.num)
    else
        say(me.name, 4)
    end
end

function step!(me::Guy, σ::Response)
    event!(𝐶, SimFunction(step!, σ.from, Greet(σ.num+1, me)), after, 2*rand())
    say(me.name, σ.num+1)
end

step! (generic function with 3 methods)

Now we need two guys (state machines), define a starting event and `run` it.

In [5]:
foo = Guy("Foo")
bar = Guy("Bar")

event!(𝐶, SimFunction(step!, foo, Meet(bar)), at, 10*rand())
run!(𝐶, 20)

 3.60 s, Foo: Nice to meet you!
 4.04 s, Bar: Nice to meet you!
 5.09 s, Foo: How are you?
 6.38 s, Bar: How are you?
 6.43 s, Foo: Have a nice day!
 7.08 s, Bar: bye bye
Finished: 6 events, simulation time: 20.0


Finally we should reset the clock for further simulations:

In [6]:
reset!(𝐶)

clock reset to t₀=0, sampling rate Δt=0.